<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# SQL Extras

## <font color=red>What Does the `CASE` Statement Do?</font>

**The SQL `CASE()` Statement** is a conditional expression, which allows me to add 'If this, then...' logic to a query.

## <font color=orange>So What Can I Do with a Case Statement?</font>

### Create Categories to Classify Values

Using CASE comes in very handy when I want to create categories from existing data based on my dataset using conditionals. 

>For example, if I have a field called `high_temp` containing integers that represent the high temperature recorded for a particular day, I can add a `CASE` statement to my query that will create a new field called `weather` classifying values in `high_temp` as one of three categorical variables, `cold`, `warm`, `hot`, based on temperature ranges that I set as conditionals.

I can create a column of categorical variables from a numeric column. I can even add a `GROUP BY` and a `COUNT(*)` to find out which category in the `weather` field has the highest count.

```sql
USE us_weather_db;
SELECT high_temp,
        COUNT(*) AS days,
    CASE high_temp > 90 THEN 'hot'
        WHEN high_temp BETWEEN 70 AND 90 THEN 'warm'
        WHEN high_temp < 70 THEN 'cold'
        ELSE 'idk'
    END AS 'weather'
FROM us_weather
GROUP BY high_temp
ORDER BY days DESC;
```
My query can have conditionals that use values from more than one field to create a new column. By throwing in a `COUNT(*) AS number_customers`, I have a useful result set for making comparisons.

```sql
USE telco_churn;
SELECT internet_service_type_id,
		device_protection,
		COUNT(*) AS number_customers,
	CASE WHEN internet_service_type_id = 1 AND device_protection = 'Yes' THEN 'DSL_Protection'
		WHEN internet_service_type_id = 1 AND device_protection = 'No' THEN 'DSL_Unprotected'
		WHEN internet_service_type_id = 2 AND device_protection = 'Yes' THEN 'Fiber_Optic_Protected'
		WHEN internet_service_type_id = 2 AND device_protection = 'No' THEN 'Fiber_Optic_Unprotected'
		WHEN internet_service_type_id = 3 THEN 'No_Internet_Service'
		ELSE internet_service_type_id
	END AS internet_types
FROM customers
GROUP BY internet_service_type_id, device_protection
ORDER BY number_customers;
```
